<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/NAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time

In [ ]:
train_dataset_path = '/content/history_train.csv'
validation_dataset_path = '/content/history_validation.csv'
test_dataset_path = '/content/history_test.csv'

train_history_data = []
validation_history_data = []
test_history_data = []

with open(train_dataset_path, 'r', newline='') as csv_train:
    csv_reader = csv.reader(csv_train)
    header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
    header = [int(item) for item in header]
    train_history_data.append(header)
    for row in csv_reader:
        r = [int(item) for item in row]
        train_history_data.append(r)

with open(validation_dataset_path, 'r', newline='') as csv_validation:
    csv_reader = csv.reader(csv_validation)
    header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
    header = [int(item) for item in header]
    validation_history_data.append(header)
    for row in csv_reader:
        r = [int(item) for item in row]
        validation_history_data.append(r)

with open(test_dataset_path, 'r', newline='') as csv_test:
    csv_reader = csv.reader(csv_test)
    header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
    header = [int(item) for item in header]
    test_history_data.append(header)
    for row in csv_reader:
        r = [int(item) for item in row]
        test_history_data.append(r)


In [ ]:
business_id_info = pd.read_csv('/content/business_info_in_philadelphia.csv' , index_col = 0)
item_list = business_id_info.index.tolist()

print(business_id_info)
print(item_list)
#print(train_history_data[0])

num_users = len(train_history_data) # 필라델피아의 사용자 수 : 15919명
num_items = business_id_info.shape[0] # 필라델피아의 가게 수 : 14603개 , 마지막 business_id : 150336

                            business_id  latitude   longitute          city
business_id_num                                                            
3                MTSW4McQd7CbVtyjqoe9mw  39.955505 -75.155564  Philadelphia
15               MUTTqe8uqyMdBl186RmNeA  39.953949 -75.143226  Philadelphia
19               ROeacJQwBeh05Rqg7F6TCg  39.943223 -75.162568  Philadelphia
28               QdN72BWoyFypdGJhhI5r7g  39.939825 -75.157447  Philadelphia
31               Mjboz24M9NlBeiOJKLEd_Q  40.022466 -75.218314  Philadelphia
...                                 ...        ...        ...           ...
150320           wPAaAbFbq5wfLfW0gnYU7g  39.950060 -75.170129  Philadelphia
150326           JDM7kiCEmeBc_5W-z6eZZw  39.968806 -75.180608  Philadelphia
150329           9U1Igcpe954LoWZRmNc-zg  39.932756 -75.144504  Philadelphia
150334           LJ4GjQ1HL6kqvIPpNUNNaQ  39.945966 -75.169666  Philadelphia
150336           WnT9NIzQgLlILjPT0kEcsQ  39.935982 -75.158665  Philadelphia

[14603 rows

In [ ]:
print(train_history_data[:3])
print(validation_history_data[:3])
print(test_history_data[:3])

[[89703, 125933, 102262, 50952, 80270, 14410, 25169, 73288, 66506, 25084], [76931, 43918, 137923, 46738, 54929, 10607, 87384, 12119, 125684, 93356], [78574, 22587, 124275, 5382, 67281, 28028, 55231, 71720]]
[[130645, 57245, 5115], [131674, 54505, 35607], [19122, 41292]]
[[51010, 41954, 54520, 41292], [52172, 69708, 52376, 43918, 132288], [56607, 110765, 110765, 8243]]


In [ ]:
def get_train_data_user(item_list, num_users, train_list, num_negatives):
    """
    학습데이터를 확인하고 num_negatives만큼의 negative데이터를 뽑아내 학습에 사용
    negative sample
    """
    num_items = len(item_list)
    user_input, item_input, labels, batch_length = [],[],[],[]

    for u in range(num_users):
       np.random.shuffle(train_list[u])
       if u == 0:
           batch_length.append((1+num_negatives) * len(train_list[u]))
       else:
           batch_length.append((1+num_negatives) * len(train_list[u])+batch_length[u-1])

       for i in train_list[u]:
            # positive instance
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
            # negative instances
            for t in range(num_negatives):
                j_idx = np.random.randint(num_items) # num_items == len(item_list), item_list에서 Negative sample을 골라야하기 때문에 인덱스를 골라줌
                j = item_list[j_idx]
                while j in train_list[u]:
                    j_idx = np.random.randint(num_items)
                    j = item_list[j_idx]
                user_input.append(u)
                item_input.append(j)
                labels.append(0)
    return user_input, item_input, labels, batch_length

def get_input_data(batch_length, train_list, user_input, item_input, batch):
    """
    recall, precision을 위한 입력 데이터 생성
    """
    p=0
    target_idx = []
    if batch == 0:
        p = 0
    else:
        p = batch_length[batch-1]
    target_idx = item_input[p:batch_length[batch]]

    temp = set(train_list[user_input[p]])

    history = []
    for idx in range(batch_length[batch]-p):
        history.append(list(temp))
    p=batch_length[batch]

    return history, target_idx


num_negatives = 4
user_input_train, item_input_train, labels_train, batch_length_train = get_train_data_user(item_list, num_users, train_history_data, num_negatives)
user_input_validation, item_input_validation, labels_validation, batch_length_validation = get_train_data_user(item_list, num_users, validation_history_data, num_negatives)

In [ ]:
print(len(user_input_train), len(item_input_train), len(labels_train), len(batch_length_train))
print(len(user_input_validation), len(item_input_validation), len(labels_validation), len(batch_length_validation))

1367085 1367085 1367085 15919
437750 437750 437750 15919


In [ ]:
class NAIS_basic(nn.Module):
    def __init__(self, item_num, embed_size, beta): # embed_size : 64, beta : 0.5
        super(NAIS_basic, self).__init__()
        self.embed_size = embed_size # concat 연산 시 * 2
        self.item_num = item_num
        self.beta = beta

        self.embed_history = nn.Embedding(item_num, embed_size) # (m * d), 과거 방문한 데이터(q), 유저별로 각각 하나씩 가져야하나 ?
        self.embed_target = nn.Embedding(item_num, embed_size) # (m * d), 예측 데이터(p)


        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.loss_func = nn.BCELoss() # binary cross entropy

        # Attention을 위한 MLP Layer 생성
        self.attn_layer1 = nn.Linear(embed_size, embed_size)
        self.attn_layer2 = nn.Linear(embed_size, 1, bias = False)

        self._init_weight_()

    def _init_weight_(self):
        # weight 초기화, 표준편차 : 0.01
        nn.init.normal_(self.embed_history.weight, std=0.01)
        nn.init.normal_(self.embed_target.weight, std=0.01)

        # bias 초기화
        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, history, target, history_region, target_region):
        history_idx = torch.LongTensor(history)
        target_idx = torch.LongTensor(target)
        prediction = self.attention_network(history_idx,target_idx)
        return self.relu(prediction)

    def attention_network(self, user_history, target_item):
        """
        b: batch size
        n: history size
        d: embedding size
        """

        history = self.embed_history(user_history) # (b * n * d)
        target = self.embed_target(target_item) # (b * 1 * d)
        batch_dim = len(target)

        target = torch.reshape(target,(batch_dim, 1,-1))
        input = history * target # (b * n * d)

        attention_result = self.relu(self.attn_layer1(input)) # (n * d)
        attention_result = self.attn_layer2(attention_result) # (n * 1)

        exp_A = torch.exp(attention_result) # (b * n * 1)
        exp_A = exp_A.squeeze(dim=-1)# (b * n)

        mask = self.get_mask(user_history,target_item)
        exp_A = exp_A * mask
        exp_sum = torch.sum(exp_A,dim=-1) # (b * 1)
        exp_sum = torch.pow(exp_sum, self.beta) # (b * 1)

        attn_weights = torch.divide(exp_A.T,exp_sum).T # (b * n)
        attn_weights = attn_weights.reshape([batch_dim,-1, 1])# (b * n * 1)
        result = history * attn_weights# (b * n * d)
        target = target.reshape([batch_dim,-1,1]) # (b * d * 1)

        prediction = torch.bmm(result, target).squeeze(dim=-1) # (b * n * 1) -> (b * n)
        prediction = torch.sum(prediction, dim = -1) # (b)
        return prediction

    def get_mask(self, user_history, target_item):
        target_item = target_item.reshape([len(target_item),1])
        mask = user_history != target_item
        return mask
    def loss_function(self, prediction, label):
        return self.loss_func(prediction, label)

In [ ]:
def train(model, epoch, user_input, item_input, labels, batch_length, trainList, optimizer, user_input_validation, item_input_validation, labels_validation, batch_length_validation):
    loss_func = nn.BCELoss()
    before_loss = 210000000.0
    max_re = 0.0

    recall = []
    prec = []
    hit = []


    for e in range(epoch):
        model.train()
        train_loss = 0.0
        start_time = int(time.time())
        p = 0

        for batch in range(len(batch_length)):
            optimizer.zero_grad()
            prediction = model(history, target_idx)
            temp = torch.tensor(labels[p:batch_length[batch]], dtype=torch.float32)
            loss = model.loss_func(prediction,temp)

            p=batch_length[batch]
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        end_time = int(time.time())
        print("Train Epoch: {}; time: {} sec; loss: {:.4f}".format(e+1, end_time-start_time,train_loss))

        re,pr,hi,re_d,pr_d,hi_d = ev.eval(model,dataset.num_users,dataset.testRatings[0], history_list,target_idx_list,history_region_idx_list,target_region_idx_list,distanceResult, 10)


        recall.append(re)
        prec.append(pr)
        hit.append(hi)

        recall_d.append(re_d)
        prec_d.append(pr_d)
        hit_d.append(hi_d)
        print("recall: {} precision: {} hit: {};".format( re[0],pr[0],hi[0]))
        re,pr,hi,re_d,pr_d,hi_d = ev.eval(model,dataset.num_users,dataset.validationList[0],history_list_val,target_idx_list_val,history_region_idx_list_val,target_region_idx_list_val,distanceResult_val, 10)

        end_time = int(time.time())
        print("time: {} sec;".format( end_time-start_time))

    return recall , prec, hit, recall_d , prec_d, hit_d
